# Cross Validation

Given that we trained and evaluated the models for each experiment with different hyperparameter combination, we are now going to perform a cross validation in the best models for each experiment, to have a more precise evaluation of the 

In [93]:
import tensorflow as tf

from numpy import genfromtxt
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection
from tensorflow import keras
from tensorflow.metrics import precision
import matplotlib.pyplot as plt 
import os
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import KFold

features = genfromtxt('../datasets/final-data/features.csv', delimiter=',')
labels = genfromtxt('../datasets/final-data/labels.csv', delimiter=',')

## Selecting the best hyperparameter combination for each experiment

In [94]:
experiments = [dI for dI in os.listdir('../experiments') if os.path.isdir(os.path.join('../experiments',dI))]

best_models = []
for experiment in experiments:
    training_metrics = pd.read_csv('../experiments/' + experiment + '/training_metrics.csv')
    training_metrics = training_metrics.sort_values(['acc', 'precision', 'recall'], ascending=[False, False, False])
    best_model = training_metrics.iloc[0]
    best_model["experiment"] = experiment
    
    best_models.append(best_model)
    
pd.DataFrame(best_models)

/Users/gcarvs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/gcarvs/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


,model_name,units,learning_rate,momentum,decay,activation_function,acc,loss,mae,mse,precision,recall,fs_score,experiment
535,model_536,35,0.0001,0.1000,0.0001,relu,0.556685,0.809226,0.5,0.314087,0.549858,0.605965,0.576550,pmean_title
344,model_345,75,0.0001,0.1000,0.1000,relu,0.548084,0.895111,0.5,0.329349,0.547812,0.530612,0.539075,pmean_body
261,model_262,75,0.0100,0.0100,0.0100,relu,0.578577,0.686320,0.5,0.262988,0.579032,0.563579,0.571201,mean_body
223,model_224,75,0.1000,0.0010,0.0001,relu,0.542611,0.736829,0.5,0.284417,0.548327,0.463108,0.502128,pmean_title_subtitle
471,model_472,35,0.0100,0.0001,0.0001,relu,0.558249,0.717674,0.5,0.275367,0.549315,0.629513,0.586686,mean_title
447,model_448,35,0.0100,0.0100,0.0001,relu,0.541830,0.704175,0.5,0.262183,0.550495,0.436421,0.486865,mean_title_subtitle


## Defining the helping functions

Here we are going to define five helper functions for the cross validation process:

1. create_model: creates a Keras model using the specified parameters
2. train_model: trains the models using the train dataset
3. test_model: tests the trained model using the test dataset
4. compute_c_validated_metrics: Computes the average and standard deviation of each metric
5. shuffle_dataset: shuffles the dataset

In [95]:
def create_model(model_name, units, activation_function, learning_rate, decay, momentum):
    linear_layer_size = 150
    if "pmean" in model_name:
        linear_layer_size = 450
    #Creating the network structure
    model = keras.Sequential()
    
    model.add(keras.layers.Input(shape=300,sparse=False))
    model.add(keras.layers.Dense(linear_layer_size))
    model.add(keras.layers.Dense(units, activation = activation_function))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(2, activation='softmax'))

    # Setting the optimizers parameters
    optimizer = tf.keras.optimizers.SGD(
        learning_rate=learning_rate,
        decay=decay,
        momentum=momentum,
        nesterov=True
    )

    # Compiling the model
    model.compile(
        optimizer = optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['acc', 'mae', 'mse'])
    
    return model

def train_model(model, train_x, train_y):
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    
    history = model.fit(
        train_x, 
        train_y, 
        epochs = 40, 
        validation_split=0.3, 
        batch_size = 16,  
        verbose=1, 
        use_multiprocessing=True,
        callbacks=[early_stop]
    )
    
    return history

def test_model(model, test_x, test_y):
    loss, acc, mae, mse = model.evaluate(test_x, test_y, verbose=1)
    
    test_output_probabilities = model.predict(
        test_x,
        batch_size=16,
        verbose=1,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=True
    )
    
    predicted_y = np.argmax(test_output_probabilities, axis=1)
    
    precision, recall, f_score, support = precision_recall_fscore_support(
        y_true = test_y, 
        y_pred = predicted_y, 
        average = 'binary'
    )
    
    return {
        "loss": loss,
        "acc": acc,
        "mae": mae,
        "mse": mse,
        "precision": precision,
        "recall": recall,
        "f_score": f_score
    }

def compute_c_validated_matrics(best_model, test_results):
    return {
        "experiment": best_model["experiment"],
        "model_name": best_model["model_name"],
        "units": best_model["units"], 
        "learning_rate": best_model["learning_rate"], 
        "momentum": best_model["momentum"],
        "decay": best_model["decay"],
        "activation_function": best_model["activation_function"],
        "average_acc": cross_validation_df["acc"].mean(),
        "acc_std_deviation": cross_validation_df["acc"].std(),
        "average_precision": cross_validation_df["precision"].mean(),
        "precision_std_deviation": cross_validation_df["precision"].std(),
        "average_recall": cross_validation_df["recall"].mean(),
        "recall_std_deviation": cross_validation_df["recall"].std(),
        "average_fs_score": cross_validation_df["f_score"].mean(),
        "fs_score_std_deviation": cross_validation_df["f_score"].std()
    }

def shuffle_dataset(features, labels):
    shuffled_indexes = np.arange(len(features))
    np.random.shuffle(shuffled_indexes)
    shuffled_features = []
    shuffled_lables = []
    for index in shuffled_indexes:
        shuffled_features.append(features[index])
        shuffled_lables.append(labels[index])
    
    return np.array(shuffled_features), np.array(shuffled_lables)

## Cross validate the models

Each model will be cross validated using 10 folders of the data, and then we will compute the cross validated metrics.

In [ ]:
cross_validated_models = []

for best_model in best_models:
    cross_validation_results = []
    
    for x in range(10):
        # Shuffle the dataset prior to initiating the validation
        features, labels = shuffle_dataset(features, labels)
        # We used k = 10 as it's the general number used in the literature
        for train_index,test_index in KFold(n_splits=10, random_state=None, shuffle=False).split(features):
            train_x, test_x = features[train_index], features[test_index]
            train_y, test_y = labels[train_index], labels[test_index]

            model = create_model(
                best_model['experiment'],
                best_model['units'], 
                best_model['activation_function'], 
                best_model['learning_rate'],
                best_model['decay'],
                best_model['momentum'])

            history = train_model(model, train_x, train_y)
            test_results = test_model(model, test_x, test_y)

            cross_validation_results.append(
                {
                    "model_name": best_model["model_name"],
                    "units": best_model["units"], 
                    "learning_rate": best_model["learning_rate"], 
                    "momentum": best_model["momentum"],
                    "decay": best_model["decay"],
                    "activation_function": best_model["activation_function"],
                    "acc": test_results["acc"],
                    "precision": test_results["precision"],
                    "recall": test_results["recall"],
                    "f_score": test_results["f_score"] 
                }
            )

    cross_validation_df = pd.DataFrame(cross_validation_results)

    cross_validated_metrics = compute_c_validated_matrics(best_model, test_results)

    cross_validated_models.append(cross_validated_metrics)

Train on 3081 samples, validate on 1321 samples
Epoch 1/40
3081/3081 [==============================] - 38s 12ms/sample - loss: 0.8178 - acc: 0.4940 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2933 - val_loss: 0.6994 - val_acc: 0.4943 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2517
Epoch 2/40
3081/3081 [==============================] - 7s 2ms/sample - loss: 0.7943 - acc: 0.5028 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2883 - val_loss: 0.7154 - val_acc: 0.4958 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2578
Epoch 3/40
3081/3081 [==============================] - 7s 2ms/sample - loss: 0.7755 - acc: 0.5060 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2845 - val_loss: 0.7527 - val_acc: 0.4951 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2724
Epoch 4/40
3081/3081 [==============================] - 7s 2ms/sample - loss: 0.7659 - acc: 0.5080 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2830 - val_

3082/3082 [==============================] - 5s 2ms/sample - loss: 0.8212 - acc: 0.5042 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2971 - val_loss: 0.8225 - val_acc: 0.4852 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2958
Epoch 5/40
3082/3082 [==============================] - 5s 2ms/sample - loss: 0.8109 - acc: 0.5026 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2939 - val_loss: 0.8243 - val_acc: 0.4875 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2962
Epoch 6/40
489/489 [==============================] - 8s 17ms/sample
Train on 3082 samples, validate on 1321 samples
Epoch 1/40
3082/3082 [==============================] - 33s 11ms/sample - loss: 0.8350 - acc: 0.4906 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3007 - val_loss: 0.6962 - val_acc: 0.5042 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2529
Epoch 2/40
3082/3082 [==============================] - 5s 2ms/sample - loss: 0.8136 - acc: 0.4951 - mean_

489/489 [==============================] - 12s 25ms/sample
Train on 3082 samples, validate on 1321 samples
Epoch 1/40
3082/3082 [==============================] - 35s 11ms/sample - loss: 0.7881 - acc: 0.5088 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2922 - val_loss: 0.6955 - val_acc: 0.5042 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2520
Epoch 2/40
3082/3082 [==============================] - 5s 2ms/sample - loss: 0.7643 - acc: 0.5182 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2859 - val_loss: 0.7097 - val_acc: 0.5072 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2595
Epoch 3/40
3082/3082 [==============================] - 6s 2ms/sample - loss: 0.7469 - acc: 0.5263 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2829 - val_loss: 0.7434 - val_acc: 0.5042 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2742
Epoch 4/40
3082/3082 [==============================] - 6s 2ms/sample - loss: 0.7405 - acc: 0.5276 - mean

3082/3082 [==============================] - 6s 2ms/sample - loss: 0.7813 - acc: 0.5042 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2885 - val_loss: 0.7524 - val_acc: 0.4928 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2728
Epoch 4/40
3082/3082 [==============================] - 6s 2ms/sample - loss: 0.7630 - acc: 0.5101 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2839 - val_loss: 0.7780 - val_acc: 0.4890 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2813
Epoch 5/40
3082/3082 [==============================] - 6s 2ms/sample - loss: 0.7579 - acc: 0.5094 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2812 - val_loss: 0.7790 - val_acc: 0.4905 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2822
Epoch 6/40
489/489 [==============================] - 9s 18ms/sample
Train on 3082 samples, validate on 1321 samples
Epoch 1/40
3082/3082 [==============================] - 37s 12ms/sample - loss: 0.7711 - acc: 0.5058 - mean_

489/489 [==============================] - 10s 20ms/sample
Train on 3082 samples, validate on 1321 samples
Epoch 1/40
3082/3082 [==============================] - 35s 11ms/sample - loss: 0.9214 - acc: 0.4912 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3188 - val_loss: 0.7076 - val_acc: 0.4905 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2544
Epoch 2/40
3082/3082 [==============================] - 6s 2ms/sample - loss: 0.8714 - acc: 0.4990 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3083 - val_loss: 0.7425 - val_acc: 0.4921 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2672
Epoch 3/40
3082/3082 [==============================] - 6s 2ms/sample - loss: 0.8453 - acc: 0.4925 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3017 - val_loss: 0.8080 - val_acc: 0.4890 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2883
Epoch 4/40
3082/3082 [==============================] - 6s 2ms/sample - loss: 0.8206 - acc: 0.4955 - mean

...

3082/3082 [==============================] - 10s 3ms/sample - loss: 0.6752 - acc: 0.5714 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2584 - val_loss: 0.6973 - val_acc: 0.5223 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2581
Epoch 5/40
3082/3082 [==============================] - 10s 3ms/sample - loss: 0.6711 - acc: 0.5857 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2585 - val_loss: 0.7020 - val_acc: 0.5284 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2606
Epoch 6/40
489/489 [==============================] - 129s 265ms/sample
Train on 3082 samples, validate on 1321 samples
Epoch 1/40
3082/3082 [==============================] - 582s 189ms/sample - loss: 0.7409 - acc: 0.4997 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2748 - val_loss: 0.6955 - val_acc: 0.4989 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2517
Epoch 2/40
3082/3082 [==============================] - 10s 3ms/sample - loss: 0.6958 - acc: 0.5425

490/490 [==============================] - 131s 267ms/sample
Train on 3081 samples, validate on 1321 samples
Epoch 1/40
3081/3081 [==============================] - 591s 192ms/sample - loss: 0.7695 - acc: 0.5018 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2775 - val_loss: 0.6954 - val_acc: 0.4913 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2510
Epoch 2/40
3081/3081 [==============================] - 11s 3ms/sample - loss: 0.6978 - acc: 0.5326 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2614 - val_loss: 0.6990 - val_acc: 0.4890 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2537
Epoch 3/40
3081/3081 [==============================] - 10s 3ms/sample - loss: 0.6818 - acc: 0.5521 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2592 - val_loss: 0.7233 - val_acc: 0.4996 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2656
Epoch 4/40
3081/3081 [==============================] - 11s 3ms/sample - loss: 0.6807 - acc: 0.5657

3082/3082 [==============================] - 1717s 557ms/sample - loss: 0.7337 - acc: 0.5078 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2695 - val_loss: 0.7002 - val_acc: 0.4951 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2528
Epoch 2/40
3082/3082 [==============================] - 11s 3ms/sample - loss: 0.6924 - acc: 0.5393 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2585 - val_loss: 0.6972 - val_acc: 0.4989 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2527
Epoch 3/40
3082/3082 [==============================] - 9s 3ms/sample - loss: 0.6820 - acc: 0.5532 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2567 - val_loss: 0.7058 - val_acc: 0.5064 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2584
Epoch 4/40
3082/3082 [==============================] - 9s 3ms/sample - loss: 0.6782 - acc: 0.5714 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2573 - val_loss: 0.7020 - val_acc: 0.5148 - val_mean_absolute_erro

3082/3082 [==============================] - 11s 3ms/sample - loss: 0.6757 - acc: 0.5798 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2638 - val_loss: 0.6969 - val_acc: 0.5163 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2592
Epoch 4/40
3082/3082 [==============================] - 9s 3ms/sample - loss: 0.6687 - acc: 0.5896 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2633 - val_loss: 0.7275 - val_acc: 0.5155 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2718
Epoch 5/40
3082/3082 [==============================] - 10s 3ms/sample - loss: 0.6604 - acc: 0.6035 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2655 - val_loss: 0.7113 - val_acc: 0.5185 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2653
Epoch 6/40
3082/3082 [==============================] - 9s 3ms/sample - loss: 0.6580 - acc: 0.6055 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2662 - val_loss: 0.7161 - val_acc: 0.5367 - val_mean_absolute_error: 0

3082/3082 [==============================] - 9s 3ms/sample - loss: 0.6720 - acc: 0.5740 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2636 - val_loss: 0.7124 - val_acc: 0.5284 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2650
Epoch 5/40
3082/3082 [==============================] - 9s 3ms/sample - loss: 0.6658 - acc: 0.5941 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2651 - val_loss: 0.7064 - val_acc: 0.5435 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2674
Epoch 6/40
3082/3082 [==============================] - 9s 3ms/sample - loss: 0.6593 - acc: 0.6009 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2659 - val_loss: 0.7218 - val_acc: 0.5413 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2711
Epoch 7/40
489/489 [==============================] - 829s 2s/sample
Train on 3082 samples, validate on 1321 samples
Epoch 1/40
3082/3082 [==============================] - 6114s 2s/sample - loss: 0.7616 - acc: 0.5097 - mean_

In [ ]:
cross_validated_df = pd.DataFrame(cross_validated_models)
cross_validated_df = cross_validated_df[
    [
        "experiment", 
        "model_name",
        "units",
        "learning_rate", 
        "momentum",
        "decay",
        "activation_function",
        "average_acc",
        "acc_std_deviation",
        "average_precision",
        "precision_std_deviation",
        "average_recall",
        "recall_std_deviation",
        "average_fs_score",
        "fs_score_std_deviation"
    ]]
cross_validated_df

In [ ]:
cross_validated_df.to_csv("../experiments/cross_validation_metrics.csv", index= False, encoding='utf-8')